In [ ]:
import serial
import numpy as np

In [ ]:
# pip3 install pyserial

1. 기본포멧
- CMD
    - 0x01: Enable Motor
    - 0x02: Set Command
    - 0x03: Request Robot States
    - 0x90 + CMD: Response for Command

- DATA
    - N bytes

- RESPONSE
    - 0x00: Don’t send Response
    - 0x01: Send Response

- LEN
    - Packet Length from CMD to DATA

- CHECKSUM
    - Total Sum from CMD to LEN
    - Casting uint8_t
---
2. Enable Motor
- DATA
    - 0x01: Enable Motor
    - 0x00: Disable Motor
---
3. Send Command
- L_VEL, R_VEL
    - 2 Bytes
    - HIGH | LOW
    - -32768 ~ -32767
    - Encoder Count / Second

- L_LED, R_LED
    - uint8_t
    - 0 ~ 255
---
4. Request States
- L_POS ,R_POS
    - Encoder Count
    - int32_t
    
- SENSOR
    - uint16_t
    - Sensor Range

In [51]:
# 파이썬에서 접근
# ser = serial.Serial('/dev/cu.usbserial-110', 1000000)
ser = serial.Serial('/dev/ttyUSB1', 1000000)
ser = serial.Serial('/dev/ttyArduino', 1000000)

print(ser.name)

/dev/ttyUSB1


In [52]:
# 한 byte씩 읽는 함수
def read(size=1, timeout=None):
    ser.timeout = timeout
    readed = ser.read(size)
    return readed

In [53]:
# 상태 받아오기
command = b"\xfa\xfe\x03\x01\x04\xfa\xfd"
ser.write(command)

incomming = read(size=20, timeout=1)
print(incomming)

b'\xfa\xfe\x93\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00p\x0e~\xfa\xfd'


In [54]:
# motor enable
command = b"\xfa\xfe\x01\x01\x01\x03\x06\xfa\xfd"
ser.write(command)

print(read(size=20, timeout=1))

b'\xfa\xfe\x91\x01\x01\x03\x8b\xfa\xfd'


In [78]:
# 모터를 돌리고 LED ON
command = [0xfa, 0xfe, 0x02, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0xfa, 0xfd]

command[3] = 0
command[4] = 5
command[5] = 0
command[6] = 5           
command[7] = 0
command[8] = 255
command[9] = 255
command[12] = np.uint8(sum(command[2:12]))

print(command)
print(bytes(command))

ser.write(bytes(command))

[250, 254, 2, 0, 5, 0, 5, 0, 255, 255, 0, 9, 19, 250, 253]
b'\xfa\xfe\x02\x00\x05\x00\x05\x00\xff\xff\x00\t\x13\xfa\xfd'


/tmp/ipykernel_19508/158313323.py:12: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 531 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  command[12] = np.uint8(sum(command[2:12]))


15